# trl 微调

[trl](https://github.com/huggingface/trl) 的功能强大，支持 SFT, PPO, DPO, GRPO 等微调方法。并且有良好的生态支持，比如，trl 可以配合 [peft](https://github.com/huggingface/peft) 的 `LoraConfig` 模块定义 LoRA 参数；配合 [unsloth](https://github.com/unslothai/unsloth) 的 `FastLanguageModel` 模型加载模型。

与上一节的 LLaMA Factory 相比，trl 可以更精细地定义训练中的行为。比如，如何加载数据集、如何构建损失函数、允许哪些参数层参与训练等等。适合需要深度控制训练过程的场景。

In [1]:
# !uv pip install --upgrade transformers
# !uv pip install bitsandbytes

In [2]:
import torch

from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

DATASET_PATH = './data/train_zh_1000.json'
MODEL_PATH = './model/Qwen/Qwen2.5-7B-Instruct/'

## 1. 加载数据集

上一节，我们将医疗数据集 [shibing624/medical](https://huggingface.co/datasets/shibing624/medical) 保存到 `data` 目录，并采样生成了 `train_zh_1000.json` 文件。

本节，我们用 `datasets` 的 `load_dataset` 方法加载 `train_zh_1000.json` 文件。关于 `load_dataset` 的数据处理逻辑，详见：[https://huggingface.co/docs/datasets/loading](https://huggingface.co/docs/datasets/loading)

In [3]:
# 从 HuggingFace 仓库加载数据集
# dataset = load_dataset("trl-lib/Capybara", split="train")

# 从本地加载数据集
dataset = load_dataset("json", data_files=DATASET_PATH)['train']

# 打印数据集的基本信息
print(f'基本信息：\n{dataset}')

# 查看数据集的行数
print(f'数据集的行数：\n{dataset.num_rows}')

# 查看数据集的形状
print(f'数据集的形状：\n{dataset.shape}')

基本信息：
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1000
})
数据集的行数：
1000
数据集的形状：
(1000, 3)


## 2. 微调 Qwen 模型

对模型使用 4-bit 量化，并用半精度浮点数加载模型。

In [4]:
# 量化配置
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

我们加载的数据集是 alpaca 格式的。下面使用 `formatting_prompts_func` 函数，将数据转换成如下格式的文本：

```
### Question:
{your_question}

### Answer:
{your_answer}
```

In [5]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

本次微调采用 `Train on completions only` 方法（仅对模型生成的 `### Answer:` 之后的部分计算损失）。什么意思呢？如果没有额外配置，我们将计算整个句子的损失，既计算 `Question` 的损失，也计算 `Answer` 的损失。这显然是不合理的。既然不用推理 `Question`，就不应该计算 `Question` 的损失。若将 `Question` 的损失加入训练，浪费算力不说，模型的优化目标也会产生偏差，导致训练效果变差。

为了达到仅计算 `Answer` 部分的损失的效果，下面用 `DataCollatorForCompletionOnlyLM` 定位样本数据 `Answer` 部分的位置。

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

我的五星级神机显存高达 8G，因此选择了较小的秩和缩放系数。如果你的 GPU 比较厉害，可以把秩和缩放系数设置得大一点，这有利于提高微调精度。比如可以设置成：

```
r=16,
lora_alpha=32,
```

`target_modules` 参数用于指定 LoRA 微调生效的模块，比较推荐微调以下模块：

- 注意力相关："q_proj", "k_proj", "v_proj", "o_proj"
- GLU 相关："gate_proj", "up_proj", "down_proj"

> 亦可参考官方文档的 `peft_config` 配置：[training-adapters](https://huggingface.co/docs/trl/sft_trainer#training-adapters)

In [7]:
# LoRA 配置
peft_config = LoraConfig(
    r=8,  # 秩
    lora_alpha=16,  # 缩放系数
    lora_dropout=0.05,  # dropout 比例
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj"
    ],  # 指定需要微调的模块
    task_type="CAUSAL_LM"
)

`SFTConfig` 的配置也压低了单卡批量数和训练轮次，因为我们旨在跑通，并非正经训练。

如果你想了解更多 `SFTConfig` 的配置详情，请参考文档：[trl.SFTConfig](https://huggingface.co/docs/trl/sft_trainer#trl.SFTConfig)

In [8]:
# SFT 训练参数
training_args = SFTConfig(
    output_dir="./Qwen2.5-0.5B-SFT",
    per_device_train_batch_size=1,  # 单卡批量数
    num_train_epochs=1,  # 训练轮次。这里仅仅为了跑通，因此设为 1
    fp16=True,  # 启用半精度训练
    optim="adamw_torch_fused",  # 使用内存优化的优化器
    max_seq_length=512,  # 序列的最大长度
    logging_steps=50,  # 日志打印间隔，默认 500
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=training_args,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator
)

trainer.train()

/home/canva/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:413: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,2.372700
100,2.149800
150,1.936900
200,2.338500
250,2.058200
300,1.975400
350,1.796700
400,2.009000
450,2.132900
500,2.244900


TrainOutput(global_step=1000, training_loss=2.0678365478515626, metrics={'train_runtime': 17254.0699, 'train_samples_per_second': 0.058, 'train_steps_per_second': 0.058, 'total_flos': 6216113574236160.0, 'train_loss': 2.0678365478515626, 'epoch': 1.0})

In [10]:
# save model
trainer.save_model()

## 3. 加载微调后的模型 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = 'cuda'

model = AutoModelForCausalLM.from_pretrained(
    "./Qwen2.5-0.5B-SFT",
    torch_dtype=torch.float16,
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-0.5B-SFT")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
def use_template(text):
    return f'### Question: {text}\n ### Answer:'

query = use_template(text='癔症有哪些表现')
inputs = tokenizer(query, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)

In [10]:
outputs

tensor([[ 14374,  15846,     25,  68294,    242,  99769, 104719, 101107,    198,
          16600,  21806,     25,    220,     16,   5373, 121998,  99769,  33071,
         113098, 119442,   5122, 101924, 100347, 115230,  57191,  99493,  99772,
         114961, 113098, 119442,   3837,  30440, 115563, 100681, 102544,   1773,
             17,   5373, 121998,  99769,  33071,  20726,  30858,   5122, 101924,
         103961, 110632, 112067, 108784,   3837,  77288, 113563, 101071,  38342,
          99879,  70633,   1773,     18,   5373, 121998,  99769,  33071]],
       device='cuda:0')

In [12]:
tokenizer.decode(outputs[0])

'### Question: 癔症有哪些表现\n ### Answer: 1、癔症性瘫痪：患者出现一侧或双侧肢体瘫痪，可伴有感觉障碍。2、癔症性失明：患者突然双眼视力丧失，但眼科检查未发现异常。3、癔症性'

参考：

- trl SFT 文档：[Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)
- trl 示例：[sft.py](https://github.com/huggingface/trl/blob/main/trl/scripts/sft.py)
- peft 文档：[peft](https://huggingface.co/docs/peft/index)
- [知乎：使用HuggingFace TRL微调Qwen1.5-7B模型（SFT）](https://zhuanlan.zhihu.com/p/692013471)
